<a href="https://colab.research.google.com/github/oorora67/AI-HandsOn/blob/master/AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Google Drive　マウント

In [0]:
#Google Drive　Mount
from google.colab import drive
drive.mount('/content/gdrive')

# **画像収集スクリプト**


In [0]:
import json
import os
import sys
import urllib
from IPython.display import Image,display_jpeg

from bs4 import BeautifulSoup
import requests
#集めたい画像の名前を入力
Search_Name = 'カレー'
#集めたい枚数を入力
Get_Number = 10

class Google:
    def __init__(self):
        self.GOOGLE_SEARCH_URL = 'https://www.google.co.jp/search'
        self.session = requests.session()
        self.session.headers.update(
            {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64; rv:10.0) Gecko/20100101 Firefox/10.0'})

    def search(self, keyword, maximum):
        print('begin searching', keyword)
        query = self.query_gen(keyword)
        return self.image_search(query, maximum)

    def query_gen(self, keyword):
        # search query generator
        page = 0
        while True:
            params = urllib.parse.urlencode({
                'q': keyword,
                'tbm': 'isch',
                'ijn': str(page)})

            yield self.GOOGLE_SEARCH_URL + '?' + params
            page += 1

    def image_search(self, query_gen, maximum):
        # search image
        result = []
        total = 0
        while True:
            # search
            html = self.session.get(next(query_gen)).text
            soup = BeautifulSoup(html, 'lxml')
            elements = soup.select('.rg_meta.notranslate')
            jsons = [json.loads(e.get_text()) for e in elements]
            imageURLs = [js['ou'] for js in jsons]

            # add search result
            if not len(imageURLs):
                print('-> no more images')
                break
            elif len(imageURLs) > maximum - total:
                result += imageURLs[:maximum - total]
                break
            else:
                result += imageURLs
                total += len(imageURLs)

        print('-> found', str(len(result)), 'images')
        return result


def main():
    google = Google()
    # save location
    name = Search_Name
    data_dir = '/content/gdrive/My Drive/AI-HandsOn/data/'
    os.makedirs(data_dir, exist_ok=True)
    os.makedirs(data_dir + name, exist_ok=True)

        # search image
    result = google.search(
    name, maximum=Get_Number)

        # download
    download_error = []
    for i in range(len(result)):
        print('-> downloading image', str(i + 1).zfill(4))
        try:
            urllib.request.urlretrieve(result[i], data_dir + name + '/' + str(i + 1).zfill(4) + '.jpg')
            display_jpeg(Image(data_dir + name + '/' + str(i + 1).zfill(4) + '.jpg'))
            continue
        except:
            print('--> could not download image', str(i + 1).zfill(4))
            download_error.append(i + 1)
            continue

    print('complete download')
    print('├─ download', len(result)-len(download_error), 'images')
    print('└─ could not download', len(download_error), 'images', download_error)


if __name__ == '__main__':
    main()